## Dialogue Summarization using LLM

In this use case, we will be generating a summary of our dialogue with the pre-trained Large Language Model (LLM) FLAN-T5 from Hugging Face.

We will use Zero-Shot, One-Shot and Few-Shot techniques. Then examine the results.

In [11]:
from datasets import load_dataset
import torch
import evaluate
import time
import pandas as pd
import numpy as np
from transformers import (AutoModelForSeq2SeqLM, AutoModelForCausalLM, 
                          AutoTokenizer, GenerationConfig, TrainingArguments, Trainer)
from transformers import AutoTokenizer
from transformers import GenerationConfig

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [3]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
torch_device = torch.device("cpu")

In [5]:
# Use the 'knkarthick/dialogsum' dataset from Hugging Face Datasets
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 309.91it/s]
Generating train split: 12460 examples [00:00, 28613.62 examples/s]
Generating validation split: 500 examples [00:00, 8676.42 examples/s]
Generating test split: 1500 examples [00:00, 29086.85 examples/s]


In [8]:
# Using two simple examples 
example_indices = [10, 70]

In [9]:
dash = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash)
    print('Example', i+1)
    print(dash)
    print('Input Dialogue: ')
    print(dataset['test'][index]['dialogue'])
    print(dash)
    print('Baseline Human Summary: ')
    print(dataset['test'][index]['summary'])
    print(dash)
    print()

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Dialogue: 
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
Basel

### FLAN- T5 Model

Flan-PaLM 540B has been fine-tuned on a large number of tasks (over 1,800 tasks) using instruction-based learning. This helps the model generalize better to unseen tasks. It achieves state-of-the-art performance on several benchmarks, such as 75.2% on five-shot MMLU. With 540 billion parameters, it's one of the largest and most powerful language models available. 


Flan models excel at dialogue summarization due to their extensive instruction-based fine-tuning, which enhances their adaptability and performance on summarization tasks. They provide high-quality summaries even with limited examples, making them efficient and effective.

In [12]:
# Using the 'google/flan-t5-base' model from Hugging Face Transformers
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name) # Load the model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True) # use_fast=True to use fast tokenization

Inference

In [13]:
# Example sentence
input_text = "Hey, can you help me with this problem?"

In [15]:
# Tokenize the input text
input_text_encoded = tokenizer(input_text, return_tensors='pt') # return_tensors='pt' to return PyTorch tensors
input_text_decoded = tokenizer.decode(input_text_encoded['input_ids'][0], skip_special_tokens=True) # skip_special_tokens=True to remove special tokens

print('Input Text:', input_text)
print(f'Encoded sentence: \n {input_text_encoded["input_ids"][0]}')
print('Decoded sentence:', input_text_decoded)

Input Text: Hey, can you help me with this problem?
Encoded sentence: 
 tensor([9459,    6,   54,   25,  199,  140,   28,   48,  682,   58,    1])
Decoded sentence: Hey, can you help me with this problem?


In [19]:
# Generate the summary
def summarize_dialogues(example_indices, dataset, prompt = "%s"): # prompt = "%s" to prompt the model with the dialogue
    for i, index in enumerate(example_indices):
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        input = prompt % dialogue # Prompt the model with the dialogue
        
        inputs = tokenizer(input, return_tensors='pt') # Tokenize the input text
        pred = model.generate(inputs['input_ids'], max_new_tokens=70)[0] # Generate the summary 
        output = tokenizer.decode(pred, skip_special_tokens=True) # Decode the summary

        print(dash)
        print('Example', i+1)
        print(dash)
        print('Input Dialogue: \n ', dialogue) 
        print(dash)
        print('Baseline Human Summary: \n', summary)
        print(dash)
        print('Model Generated Summary: \n', output)
              

In [20]:
summarize_dialogues(example_indices, dataset)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Dialogue: 
  #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
Bas

The model shows room for improvement as the generated summaries could be more detailed and precise. While it does a fair job for example 2, its performance for example 1 can be more accurate and coherent.

### Zero Shot Inference with an Instruction Prompt

In [24]:
prompt = "Summarize the following conversation.%s" # Prompt the model with the dialogue 
print(prompt)

Summarize the following conversation.%s


In [25]:
summarize_dialogues(example_indices, dataset, prompt)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Dialogue: 
  #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
Bas

In [ ]:
prompt = "What happened?%s" 
print(prompt)

What happened?%s


In [29]:
summarize_dialogues(example_indices, dataset, prompt)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Dialogue: 
  #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
Bas

The results are slightly better after we gave it some prompts. But it can still be improved.

### One Shot Inference

In [40]:
# Function to generate the prompt
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        prompt += f"""Dialogue:\n{dialogue}\n\nWhat was going on?\n{summary}\n\n\n"""
        dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f'Dialogue:\n{dialogue}\n\nWhat was going on?'
    return prompt

In [47]:
#Giving example to model and giving a dialogue which needs to be summarized
example_indices_full = [10]
example_index_to_summarize = 80

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

Dialogue:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

What was going on?
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.


Dialogue:
#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is 

In [39]:
# Generate the summary
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True
)

print(dash)
print(f'Baseline Human Summary: \n{summary}\n')
print(dash)
print(f'Model Generation - One Shot:\n{output}')

---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
Mom asks May to help to prepare for the picnic and May agrees.

---------------------------------------------------------------------------------------------------
Model Generation - One Shot:
Person1 wants to prepare for the picnic. May will help her prepare the food. Daniel will help her.


Now let's try to further improve our model by giving the model multiple examples.

### Few Shot Inference

In [52]:
# Giving example to model and giving a dialogue which needs to be summarized
example_indices_full = [2, 5, 12]
example_index_to_summarize = 70

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

Dialogue:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with th

In [53]:
# Generate the summary
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True
)

print(dash)
print(f'Baseline Human Summary: \n{summary}\n')
print(dash)
print(f'Model Generation - Few Shot:\n{output}')

---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# tells Tom that his novel has won the Nobel Prize.

---------------------------------------------------------------------------------------------------
Model Generation - Few Shot:
Tom's novel has won the Nobel Prize.
